# Temperature - Basic Usage of HiGlass

In this notebook we will talk through the main aspects of the API and how to get started visualizing data

In [1]:
%load_ext autoreload
%autoreload 2

### Load data

We optained the data from the National Centers for Environmental Information: https://www.ncei.noaa.gov/access/search/data-search/global-hourly

In [2]:
from utils import get_ncei_temperature_data_as_array

temps_by_city, min_max_dates = get_ncei_temperature_data_as_array({
    "corpus_christi": {
        "files": ["data/1796105.csv.gz"],
        "station": 72251012924
    },
    "boston": {
        "files": [
            "data/1577451.csv.gz",
            "data/1577468.csv.gz",
            "data/1573178.csv.gz"
        ],
        "station": 72509014739
    },
    "fairbanks": {
        "files": ["data/1707246.csv.gz"],
        "station": 70261026411
    }
})

In [3]:
import numpy as np

print('Temperature measurements from {} to {}'.format(*min_max_dates))

print('Temperatures in Corpus Christi range from {}°C to {}°C'.format(
    np.nanmin(temps_by_city['corpus_christi']), np.nanmax(temps_by_city['corpus_christi'])
))

print('Temperatures in Boston range from {}°C to {}°C'.format(
    np.nanmin(temps_by_city['boston']), np.nanmax(temps_by_city['boston'])
))

print('Temperatures in Fairbanks range from {}°C to {}°C'.format(
    np.nanmin(temps_by_city['fairbanks']), np.nanmax(temps_by_city['fairbanks'])
))

Temperature measurements from 1953-01-01 00:00:00 to 2018-01-01 23:54:00
Temperatures in Corpus Christi range from -15.5°C to 48.9°C
Temperatures in Boston range from -24.4°C to 38.9°C
Temperatures in Fairbanks range from -52.2°C to 34.4°C


### Create tilesets

In [29]:
from higlass.tilesets import Tileset
from clodius.tiles.npvector import tileset_info, tiles_wrapper

# Sorry we need a micro fix 😬
def fixed_tileset_info(uuid):
    info = tileset_info(uuid)
    info['tile_size'] = info['bins_per_dimension']
    return info

tileset_corpus_christi = Tileset(
    tileset_info=lambda: fixed_tileset_info(temps_by_city['corpus_christi']),
    tiles=lambda tile_ids: tiles_wrapper(temps_by_city['corpus_christi'], tile_ids),
    name='Corpus Christi'
)

tileset_boston = Tileset(
    tileset_info=lambda: fixed_tileset_info(temps_by_city['boston']),
    tiles=lambda tile_ids: tiles_wrapper(temps_by_city['boston'], tile_ids),
    name='Boston'
)

tileset_fairbanks = Tileset(
    tileset_info=lambda: fixed_tileset_info(temps_by_city['fairbanks']),
    tiles=lambda tile_ids: tiles_wrapper(temps_by_city['fairbanks'], tile_ids),
    name='Fairbanks'
)

### Create HiGlass view

In [5]:
import higlass
from higlass.client import Track, View

top_axis = Track('top-axis')

track_definition = {
    'track_type': 'horizontal-line',
    'height': 100
}

corpus_christi_line = Track(tileset=tileset_corpus_christi, **track_definition)
boston_line = Track(tileset=tileset_boston, **track_definition)
fairbanks_line = Track(tileset=tileset_fairbanks, **track_definition)

display, server, viewconf = higlass.display([
    View([top_axis, corpus_christi_line, boston_line, fairbanks_line])
])

display

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'HLV58RkBRgy4E…

### Customize appearance of tracks

In [6]:
from copy import deepcopy

## Important change ##########
new_track_definition = {
    'options': {
        'labelPosition': 'hidden',
        'axisPositionHorizontal': 'hidden',
        'lineStrokeWidth': 2,
        'valueScaleMin': -50,
        'valueScaleMax': +50,
        'constIndicators': [
            { 'value':   0, 'label':   '0', 'labelSize': 8, 'opacity': 0.3 },
            { 'value':  20, 'label':  '20', 'labelSize': 8, 'opacity': 0.3 },
            { 'value': -20, 'label': '-20', 'labelSize': 8, 'opacity': 0.3 },
            { 'value':  40, 'label':  '40', 'labelSize': 8, 'opacity': 0.3 },
            { 'value': -40, 'label': '-40', 'labelSize': 8, 'opacity': 0.3 },
        ],
    }
}
new_track_definition.update(track_definition)
##############################

track_def_cc = deepcopy(new_track_definition)
track_def_cc['options']['lineStrokeColor'] = 'orange'
corpus_christi_line = Track(tileset=tileset_corpus_christi, **track_def_cc)

track_def_bos = deepcopy(new_track_definition)
track_def_bos['options']['lineStrokeColor'] = 'mediumseagreen'
boston_line = Track(tileset=tileset_boston, **track_def_bos)

track_def_fb = deepcopy(new_track_definition)
track_def_fb['options']['lineStrokeColor'] = 'blue'
fairbanks_line = Track(tileset=tileset_fairbanks, **track_def_fb)

display, server, viewconf = higlass.display([
    View([top_axis, corpus_christi_line, boston_line, fairbanks_line])
])

display

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'Bw3aBEyrQhWKY…

### Combine tracks

In [7]:
from higlass.client import CombinedTrack

## Important change ##########
combined_track = CombinedTrack(
    position='top',
    height=200,
    tracks=[corpus_christi_line, boston_line, fairbanks_line]
)
##############################

display, server, viewconf = higlass.display([
    View([top_axis, combined_track])
])

display

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'PbE7EIh0QFWQt…

### Adjust view

In [20]:
corpus_christi_line = Track(tileset=tileset_corpus_christi, **track_def_cc)
boston_line = Track(tileset=tileset_boston, **track_def_bos)
fairbanks_line = Track(tileset=tileset_fairbanks, **track_def_fb)

combined_track = CombinedTrack(
    position='top',
    height=200,
    tracks=[corpus_christi_line, boston_line, fairbanks_line]
)

## Important change ##########
view_definition = {
    'initialXDomain': [max_size * 0.48, max_size * 0.52],
    'tracks': [top_axis, combined_track]
}
##############################

display, _, _ = higlass.display([View(**view_definition)])

display

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'b4JGm_tkQ4KoT…

### Multiple Views

In [40]:
heatmap_settings = {
    'track_type': 'horizontal-1d-heatmap',
    'position': 'top',
    'height': 48,
    'options': {
        'colorRange': ['#003aa8', '#3379ff', '#dddddd', '#ff3d33', '#990700'],
        'valueScaleMin': -20,
        'valueScaleMax': +20,
    }
}

corpus_christi_heatmap = Track(tileset=tileset_corpus_christi, **heatmap_settings)
boston_heatmap = Track(tileset=tileset_boston, **heatmap_settings)
fairbanks_heatmap = Track(tileset=tileset_fairbanks, **heatmap_settings)

max_size = temps_by_city['corpus_christi'].size

## Important change ##########
view_a = {
    'x': 0,
    'y': 0,
    'initialXDomain': [540000, 566000],
    'tracks': [top_axis, corpus_christi_heatmap, boston_heatmap, fairbanks_heatmap]
}
view_b = {
    'x': 0,
    'y': 1,
    'initialXDomain': [540000, 566000],
    'tracks': [top_axis, combined_track]
}
##############################

display, _, viewconf = higlass.display([View(**view_a), View(**view_b)])

display

HiGlassDisplay(hg_options={'isDarkTheme': False}, viewconf={'editable': True, 'views': [{'uid': 'ZUVGLTGuTt-vH…

☝️ Winters 2014/15, 2015/16, and 2016/17